# Notebook pour le calcul des indices NDWI, NDVI (LANDSAT-S2)

### Initialisation contexte test appli

In [ ]:
import os

# Manually filling Config dictionnary (can be filled by reading Config_process.env) :

CONFIG = {}

CONFIG['WRK_DIR'] = os.path.normpath('D:/MATHIS/0_Projet_Secheresse/1_Scripts/toolbox/eo4dm-oeil/EO4DM')
os.chdir(CONFIG['WRK_DIR'])
CONFIG['WRK_DIR'] = os.path.normpath('Y:/EO4DM')
CONFIG['DATA_HISTO'] = os.path.join(CONFIG['WRK_DIR'],'DATA_HISTO')
CONFIG['ANNEX_DIR'] = os.path.join(CONFIG['WRK_DIR'],'ANNEX')

CONFIG['TERRITORY'] = 'New Caledonia (Fr)'
CONFIG['PERIOD_START'] = '2023-11-11'
CONFIG['PERIOD_END'] = '2023-11-21'
CONFIG['DRIVE_FOLDER'] = 'EO4DM_EXPORT_NOTEBOOK'
CONFIG['LANDMASK_ROI'] = 0                                # if 1, preproc products are masked for pixels outside landmask roi (input shapefile to upload on gee asset)
CONFIG['ASSET_EXPORT_L'] = 0                                # if 1, preproc products are exported to gee
CONFIG['ASSET_EXPORT_S2'] = 0                                # if 1, preproc products are exported to gee
CONFIG['CLEAN_GEEFOLDER'] = 0                             # if 1, gee already existing folder is cleaned (deleted, and re-created)
CONFIG['CLEAN_GEECOL'] = 0                                # if 1, gee already exported products are cleaned (deleted, and re-created)
CONFIG['CLEAN_RUNFOLDER'] = 0                         # if 1, drought already existing folder is cleaned (deleted, and re-created)
CONFIG['MODE'] = 'DROUGHT'                                 # 'AUTO', 'MANUAL', 'INDICES' or 'DROUGHT' mode of drought processing chain

CONFIG['TILES_L'] = ['82075','81075'] # fill with '' (if empty)
# CONFIG['TILES_L'] = ['82075','81075','83074','82074','81074','80075','81076'] # fill with '' (if empty)
CONFIG['TILES_S2'] = ['58KCC','58KCD','58KDB','58KDC']
# CONFIG['TILES_S2'] = ['58KCC','58KCD','58KDB','58KDC','58KEA','58KEB','58KEC','58KFA','58KFB','58KFC','58KGA','58KGB','58KGC','58KGV','58KHB']

### Test détection auto de la période à traiter (en fonction de data_histo)

In [ ]:
import dmpipeline.DROUGHT_Processing.DROUGHT_local_functions as dloc
go_process, CONFIG['PERIOD_START'], CONFIG['PERIOD_END'] = dloc.auto_processingPERIOD(CONFIG)

print(f'go_process = {go_process}')
PERIOD_START = CONFIG['PERIOD_START']
PERIOD_END = CONFIG['PERIOD_END']
print(f'PERIOD_START = {PERIOD_START}')
print(f'PERIOD_END = {PERIOD_END}')

In [ ]:
import dmpipeline.GEE_Processing.GEE_main_functions as geemain
go_landsat_s2, go_month, PERIOD_START, PERIOD_END, TERRITORY, TILES_L, TILES_S2 = geemain.check_GEEProducts_LANDSAT_S2(CONFIG)

print(f'go_landsat_s2 = {go_landsat_s2}')
print(f'go_month = {go_month}')
print(f'PERIOD_START = {PERIOD_START}')
print(f'PERIOD_END = {PERIOD_END}')
print(f'TERRITORY = {TERRITORY}')
print(f'TILES_L = {TILES_L}')
print(f'TILES_S2 = {TILES_S2}')

In [ ]:
import dmpipeline.DROUGHT_Processing.DROUGHT_local_functions as dloc
OUTDIR_PATHS = dloc.prepare_RUNFolders(CONFIG)

In [ ]:
import dmpipeline.GEE_Processing.GEE_main_functions as geemain
COLLECTIONS, PARAM, DICT = geemain.initialize_GEEProcessing_LANDSAT_S2(CONFIG, OUTDIR_PATHS[3])

### Vérification bonne lecture GEE des collections LANDSAT/S2

In [ ]:
# (LANDSAT_COLLECTIONS, S2_COLLECTIONS,
#  NEW_LANDSATCOLLECTIONS, NEW_S2COLLECTIONS, LANDSAT_GRIDS, landmask) = COLLECTIONS
# (SCALE_OUT, DATES) = PARAM

# tile_L = CONFIG['TILES_L'][0]

# print(f'\n --- TILE LANDSAT {tile_L} :')

# (L7_tile, L8_tile, L9_tile) = LANDSAT_COLLECTIONS[tile_L]
# S2_alltiles = S2_COLLECTIONS[tile_L]

# Nb_L7images = L7_tile.size().getInfo()
# Nb_L8images = L8_tile.size().getInfo()
# Nb_L9images = L9_tile.size().getInfo()
# Nb_S2images = S2_alltiles.size().getInfo()

# (date_start, date_end) = DATES[tile_L]
# date_start_str = date_start.format('YYYYMMdd').getInfo()
# date_end_str = date_end.format('YYYYMMdd').getInfo()

# print(f'     {date_start_str} -> {date_end_str}')
# print(f'     - L7 = {Nb_L7images}')
# print(f'     - L8 = {Nb_L8images}')
# print(f'     - L9 = {Nb_L9images}')
# print(f'     - S2 = {Nb_S2images}')

# print(f'NEW GEE LANDSAT Collection path:\n{NEW_LANDSATCOLLECTIONS[tile_L]}\n')

# print(f'NEW GEE S2 Collection paths:')
# if Nb_S2images==0:  print('No path (no S2 collections on the selected area/period)')
# else:
#   for tile_S2 in NEW_S2COLLECTIONS[tile_L]:
#     print(f'   - {tile_S2}: {NEW_S2COLLECTIONS[tile_L][tile_S2]}')


### Extrait collections pour la tuile Landast tile_L, un mois m (et une année y)

In [ ]:
# import ee
# import dmpipeline.GEE_Processing.gee_accounts as geeauth

# path2key = os.path.dirname(geeauth.__file__)
# proj = L7_tile.merge(L8_tile).merge(L9_tile).first().select('SR_B4').projection().getInfo()

# (QA_dict_L, QAtable_filename_L, QAtable_columns_L,
#  QA_dict_S2, QAtable_filename_S2, QAtable_columns_S2,
#  COMP_dict, COMPtable_filename, COMPtable_columns) = DICT
# # y=2002
# # m=7
# y = ee.Date(date_start).get('year').getInfo()
# m = ee.Date(date_start).get('month').getInfo()

# L7_month = (L7_tile
#             .filter(ee.Filter.calendarRange(y,y,'year'))
#             .filter(ee.Filter.calendarRange(m,m,'month')))
# L8_month = (L8_tile
#             .filter(ee.Filter.calendarRange(y,y,'year'))
#             .filter(ee.Filter.calendarRange(m,m,'month')))
# L9_month = (L9_tile
#             .filter(ee.Filter.calendarRange(y,y,'year'))
#             .filter(ee.Filter.calendarRange(m,m,'month')))
# S2_month = (S2_alltiles
#             .filter(ee.Filter.calendarRange(y,y,'year'))
#             .filter(ee.Filter.calendarRange(m,m,'month')))


### Test Calcul indices LANDSAT-7 sur deux dates

In [ ]:
# import dmpipeline.GEE_Processing.GEE_preprocessing_functions as geeprep

# L7_list = L7_month.toList(L7_month.size().getInfo())

# # Extrait L7 pour 1er produit du mois
# i = 0
# l7 = ee.Image(L7_list.get(i))
# l7_indices_preproc, QA_dict_L  = geeprep.preprocessingL7(l7, landmask, LANDSAT_GRIDS[tile_L], tile_L, NEW_LANDSATCOLLECTIONS[tile_L], QA_dict_L, path2key, int(CONFIG['ASSET_EXPORT']))

# # Extrait L7 pour 2ème produit du mois
# i = 1
# l7 = ee.Image(L7_list.get(i))
# l7_indices_preproc_2, QA_dict_L  = geeprep.preprocessingL7(l7, landmask, LANDSAT_GRIDS[tile_L], tile_L, NEW_LANDSATCOLLECTIONS[tile_L], QA_dict_L, path2key, int(CONFIG['ASSET_EXPORT']))


In [ ]:
# # Import already processed assets (to avoid recomputing, if needed)
# import ee
# l7_indices_preproc = ee.Image('projects/eo4dm-367504/assets/PREPROC_LOCAL_INDICES_New_Caledonia_Fr/LANDSAT_82074/LE07_L2SR_082074_20090509_20200912_02_T1')
# l7_indices_preproc_2 = ee.Image('projects/eo4dm-367504/assets/PREPROC_LOCAL_INDICES_New_Caledonia_Fr/LANDSAT_82074/LE07_L2SR_082074_20090525_20200912_02_T1')

### Test Composition temporelle indices LANDSAT-7 sur deux dates

In [ ]:
# import dmpipeline.GEE_Processing.GEE_compositing_functions as geecomp
# import dmpipeline.GEE_Processing.GEE_generic_functions as geegen

# # Concatenates both images into a single image collection
# indices_comp = ee.ImageCollection([l7_indices_preproc, l7_indices_preproc_2])

# # Computes composite on each band
# INDICES_comp = geecomp.extractComposite(indices_comp, LANDSAT_GRIDS[tile_L])
# NDWI_comp = INDICES_comp['NDWI']
# NDVI_comp = INDICES_comp['NDVI']

# band_ndwi_comp = NDWI_comp.bandNames().getInfo()
# band_ndvi_comp = NDVI_comp.bandNames().getInfo()

# crs_ndwi_comp = NDWI_comp.projection().getInfo()['crs']
# crs_ndvi_comp = NDVI_comp.projection().getInfo()['crs']

# print(f'\nbands comp NDWI = {band_ndwi_comp}')
# print(f'\nbands comp NDVI = {band_ndvi_comp}')
# print(f'\ncrs comp NDWI = {crs_ndwi_comp}')

In [ ]:
# composite_image = INDICES_comp['NDWI'].addBands(INDICES_comp['NDVI'], overwrite=True).select(['NDWI', 'NDVI'])

# # Check geometries (after and before compositing)
# l7_geom = l7_indices_preproc.select('NDWI').geometry().getInfo()['coordinates']
# l7_geom_2 = l7_indices_preproc_2.select('NDWI').geometry().getInfo()['coordinates']
# comp_geom = composite_image.select('NDWI').geometry().getInfo()['coordinates']

# print(f'l7_geom = {l7_geom}')
# print(f'l7_geom dip1 = {l7_geom_2}')
# print(f'comp_geom = {comp_geom}')

In [ ]:
# # Extracts nan scores :
# l7_NANSCORES = geegen.computeNanScores(l7_indices_preproc, landmask)
# l7_NANSCORES_2 = geegen.computeNanScores(l7_indices_preproc_2, landmask)
# print(f'\nl7 nan scores ori = {l7_NANSCORES}')
# print(f'\nl7 nan scores dip1 ori = {l7_NANSCORES_2}')

# l7_NANSCORES_new = geegen.computeNanScores(l7_indices_preproc, landmask, data_geom=LANDSAT_GRIDS[tile_L])
# l7_NANSCORES_new_2 = geegen.computeNanScores(l7_indices_preproc_2, landmask, data_geom=LANDSAT_GRIDS[tile_L])
# print(f'\nl7 nan scores new = {l7_NANSCORES_new}')
# print(f'\nl7 nan scores dip1 new = {l7_NANSCORES_new_2}')

# COMP_NANSCORES = geegen.computeNanScores(composite_image, landmask, data_geom=LANDSAT_GRIDS[tile_L], scale=SCALE_OUT)
# print(f'\ncomposite nan scores = {COMP_NANSCORES}')


In [ ]:
# import dmpipeline.GEE_Processing.GEE_generic_functions as geegen

# comp_ndwi_filename = 'LANDSAT_7_NDWI'
# comp_ndvi_filename = 'LANDSAT_7_NDVI'

# # geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['NDWI'], comp_ndwi_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=proj['crs'], data_scale=SCALE_OUT, data_region=LANDSAT_GRIDS[tile_L])
# geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['NDVI'], comp_ndvi_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=proj['crs'], data_scale=SCALE_OUT, data_region=LANDSAT_GRIDS[tile_L])

### Test Calcul indices LANDSAT/S2 (un indice par capteur disponible)

In [ ]:

# import dmpipeline.GEE_Processing.GEE_preprocessing_functions as geeprep

# if L7_month.size().getInfo()!=0:
#   l7_indices_preproc, QA_dict_L  = geeprep.preprocessingL7(L7_month.first(), landmask, LANDSAT_GRIDS[tile_L], tile_L, NEW_LANDSATCOLLECTIONS[tile_L], QA_dict_L, path2key, int(CONFIG['ASSET_EXPORT']))

# if L8_month.size().getInfo()!=0:
#   l8_indices_preproc, QA_dict_L = geeprep.preprocessingL8L9(L8_month.first(), landmask, LANDSAT_GRIDS[tile_L], tile_L, NEW_LANDSATCOLLECTIONS[tile_L], QA_dict_L, path2key, int(CONFIG['ASSET_EXPORT']))

# if L9_month.size().getInfo()!=0:
#   l9_indices_preproc, QA_dict_L = geeprep.preprocessingL8L9(L9_month.first(), landmask, LANDSAT_GRIDS[tile_L], tile_L, NEW_LANDSATCOLLECTIONS[tile_L], QA_dict_L, path2key, int(CONFIG['ASSET_EXPORT']))

# if S2_month.size().getInfo()!=0:
#   tile_S2 = S2_month.get('MGRS_TILE').getInfo()
#   s2_indices_preproc, QA_dict_S2 = geeprep.preprocessingS2(S2_month.first(), landmask, tile_L, NEW_S2COLLECTIONS[tile_L][tile_S2], QA_dict_S2, path2key, int(CONFIG['ASSET_EXPORT']))


In [ ]:
# if L7_month.size().getInfo()!=0:
#   bandsl7 = l7_indices_preproc.bandNames().getInfo()
#   QA_l7 = QA_dict_L[0].getInfo()['properties']
#   print(f'bandsl7 = {bandsl7}')
#   print(f'QA_l7 : \n{QA_l7}')

# if L8_month.size().getInfo()!=0:
#   bandsl8 = l8_indices_preproc.bandNames().getInfo()
#   QA_l8 = QA_dict_L[0].getInfo()['properties']
#   print(f'bandsl8 = {bandsl8}')
#   print(f'QA_l8 : \n{QA_l8}')

# if L9_month.size().getInfo()!=0:
#   bandsl9 = l9_indices_preproc.bandNames().getInfo()
#   QA_l9 = QA_dict_L[1].getInfo()['properties']
#   print(f'bandsl9 = {bandsl9}')
#   print(f'QA_l9 : \n{QA_l9}')

# if S2_month.size().getInfo()!=0:
#   bandss2 = s2_indices_preproc.bandNames().getInfo()
#   QA_s2 = QA_dict_S2[0].getInfo()['properties']
#   print(f'bandss2 = {bandss2}')
#   print(f'QA_s2 : \n{QA_s2}')


### Test Composition temporelle indices LANDSAT/S2

In [ ]:
# /!\ A ADAPTER POUR LANDSAT/S2... /!\

# # Load already processed indices (to avoid running again above lines)
# import ee
# import GEE_Processing.GEE_generic_functions as geegen

# project_id = geegen.googleAuthentification(os.path.join(os.getcwd(),'GEE_Processing'))
# new_collection = 'projects/eo4dm-367504/assets/PREPROC_GLOBAL_INDICES_200301_200301_New_Caledonia_Fr/MODIS'

# di = 1
# indices_preproc = ee.ImageCollection(new_collection).filter(ee.Filter.calendarRange(di,di,'DAY_OF_MONTH')).first()

# dip1 = 2
# indices_preproc = ee.ImageCollection(new_collection).filter(ee.Filter.calendarRange(dip1,dip1,'DAY_OF_MONTH')).first()

# GRID_OUT =  ee.Geometry.Polygon([[[BBOX_OUT['lon_min'], BBOX_OUT['lat_max']],
#                                   [BBOX_OUT['lon_min'], BBOX_OUT['lat_min']],
#                                   [BBOX_OUT['lon_max'], BBOX_OUT['lat_min']],
#                                   [BBOX_OUT['lon_max'], BBOX_OUT['lat_max']]]],None, False)

# SCALE_REFLECT = new_ndwi_di.select('NDWI').projection().nominalScale()

# print(indices_preproc.bandNames().getInfo())
# print(indices_preproc.date().format('YYYYMMdd').getInfo())
# print(indices_preproc_dip1.bandNames().getInfo())
# print(indices_preproc_dip1.date().format('YYYYMMdd').getInfo())


In [ ]:
# Concatenates both images into a single image collection
# /!\ A MODIFIER SELON INDICES EN ENTREE
# indices_comp = ee.ImageCollection([l8_indices_preproc, l9_indices_preproc, s2_indices_preproc])

In [ ]:
# import dmpipeline.GEE_Processing.GEE_compositing_functions as geecomp

# # Computes composite on each band
# INDICES_comp = geecomp.extractComposite(indices_comp, LANDSAT_GRIDS[tile_L])


In [ ]:
# NDWI_comp = INDICES_comp['NDWI']
# NDVI_comp = INDICES_comp['NDVI']

# band_ndwi_comp = NDWI_comp.bandNames().getInfo()
# band_ndvi_comp = NDVI_comp.bandNames().getInfo()

# crs_ndwi_comp = NDWI_comp.projection().getInfo()['crs']
# crs_ndvi_comp = NDVI_comp.projection().getInfo()['crs']

# print(f'\nbands comp NDWI = {band_ndwi_comp}')
# print(f'\nbands comp NDVI = {band_ndvi_comp}')

# print(f'\ncrs comp NDWI = {crs_ndwi_comp}')
# print(f'\ncrs comp NDVI = {crs_ndvi_comp}')

In [ ]:
# import dmpipeline.GEE_Processing.GEE_generic_functions as geegen

# comp_ndwi_filename = 'LANDSAT_SENTINEL2_NDWI'
# comp_ndvi_filename = 'LANDSAT_SENTINEL2_NDVI'

In [ ]:
# geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['NDWI'], comp_ndwi_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=proj['crs'], data_transform=proj['transform'], data_region=LANDSAT_GRIDS[tile_L])
# geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['NDVI'], comp_ndvi_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=proj['crs'], data_transform=proj['transform'], data_region=LANDSAT_GRIDS[tile_L])

### Composition temporelle des indices LANDSAT/S2 par décade (sur un mois)

In [ ]:
# import dmpipeline.GEE_Processing.GEE_compositing_functions as geecomp

# # Calcul composites décades

# (NDWIcomp_col, NDVIcomp_col, QA_dict_L, QA_dict_S2,COMP_TYPES,
#  COMP_NANSCORES, NBL8_DATES, NBL9_DATES, NBS2_DATES) = geecomp.processCompositeDecade_LANDSAT_S2(L8_tile, L9_tile, S2_alltiles,
#                                                                                                   y, m, landmask, LANDSAT_GRIDS[tile_L], tile_L, SCALE_OUT,
#                                                                                                   NEW_LANDSATCOLLECTIONS[tile_L], NEW_S2COLLECTIONS[tile_L],
#                                                                                                   QA_dict_L, QA_dict_S2, path2key, int(CONFIG['ASSET_EXPORT']))

In [ ]:
# import time
# import dmpipeline.GEE_Processing.GEE_generic_functions as geegen

# # Exporte composites
# month2find = '{}{:02d}'.format(y,m)
# N_dec = NDWIcomp_col.size().getInfo()
# NDWIcomp_list = NDWIcomp_col.toList(N_dec)
# NDVIcomp_list = NDVIcomp_col.toList(N_dec)

# for d in range(1,N_dec):#range(N_dec):
#   INDICES_comp = {}
#   INDICES_comp['NDWI'] = ee.Image(NDWIcomp_list.get(d))
#   INDICES_comp['NDVI'] = ee.Image(NDVIcomp_list.get(d))
  
#   comp_ndwi_filename = f'LANDSAT_SENTINEL2_{tile_L}_{month2find}_NDWI_{COMP_TYPES[d]}'
#   comp_ndvi_filename = f'LANDSAT_SENTINEL2_{tile_L}_{month2find}_NDVI_{COMP_TYPES[d]}'

#   start_time = time.time()
#   geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['NDWI'], comp_ndwi_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=proj['crs'], data_scale=SCALE_OUT, data_region=LANDSAT_GRIDS[tile_L])
#   geegen.exportImage(CONFIG['DRIVE_FOLDER'], INDICES_comp['NDVI'], comp_ndvi_filename, export_folder=OUTDIR_PATHS[1], path2key=path2key, data_crs=proj['crs'], data_scale=SCALE_OUT, data_region=LANDSAT_GRIDS[tile_L])  
#   elapsed_time = round(time.time() - start_time)

#   COMP_dict += [ee.Feature(None, {'TILE': tile_L,
#                                   'DATE': month2find,
#                                   'COMPOSITE': COMP_TYPES[d],
#                                   'L7': 0,
#                                   'L8': NBL8_DATES[d],
#                                   'L9': NBL9_DATES[d],
#                                   'S2': NBS2_DATES[d],
#                                   'COMPOSITE NAN SCORE NDWI': COMP_NANSCORES[d]['NDWI'],
#                                   'COMPOSITE NAN SCORE NDVI': COMP_NANSCORES[d]['NDVI'],
#                                   'COMPOSITE TIME (sec)': elapsed_time})]
#   del start_time, elapsed_time, INDICES_comp, comp_ndwi_filename, comp_ndvi_filename


In [ ]:
# # --- SAVE TABLES (DATA FRAMES) INTO CSV FILES (LOCAL MACHINE) ---
# geegen.exportDataFrame(CONFIG['DRIVE_FOLDER'], QA_dict_L, QAtable_filename_L, QAtable_columns_L, OUTDIR_PATHS[3], path2key)
# geegen.exportDataFrame(CONFIG['DRIVE_FOLDER'], QA_dict_S2, QAtable_filename_S2, QAtable_columns_S2, OUTDIR_PATHS[3], path2key)
# geegen.exportDataFrame(CONFIG['DRIVE_FOLDER'], COMP_dict, COMPtable_filename, COMPtable_columns, OUTDIR_PATHS[3], path2key)

### Test complet avec fonction Run LANDSAT/S2 (intègre différents traitements précédents)

In [ ]:
geemain.run_GEEProcessing_LANDSAT_S2(CONFIG, OUTDIR_PATHS, COLLECTIONS, PARAM, DICT)